In [19]:
import utils as utils
import numpy as np
import json

In [2]:
with open('3k5d.pdb') as f:
    pdb = f.read()

In [3]:

model = utils.getModel(pdb)

In [15]:
def plane_normal(p1, p2, p3):
    """Calculate the normal vector of a plane defined by three points."""
    v1 = np.array(p2) - np.array(p1)
    v2 = np.array(p3) - np.array(p2)
    normal = np.cross(v1, v2)  # Cross product to get the normal
    return normal / np.linalg.norm(normal)  # Normalize the normal vector

def angle_between_planes(plane1, plane2):
    """Compute the angle (in degrees) between two planes using their normal vectors."""
    dot_product = np.dot(plane1, plane2)
    angle_rad = np.arccos(np.clip(dot_product, -1.0, 1.0))  # Avoid numerical errors
    return np.degrees(angle_rad)

def get_dihedral_angles(res_1, res_2):
    if res_1 is None or res_2 is None: return None, None, None
    normal_1 = plane_normal(res_1.atoms['N'][1], res_1.atoms['CA'][1], res_1.atoms['C'][1])
    normal_2 = plane_normal(res_1.atoms['CA'][1], res_1.atoms['C'][1], res_2.atoms['N'][1])
    normal_3 = plane_normal(res_1.atoms['C'][1], res_2.atoms['N'][1], res_2.atoms['CA'][1])
    normal_4 = plane_normal(res_2.atoms['N'][1], res_2.atoms['CA'][1], res_2.atoms['C'][1])

    psi_1 = angle_between_planes(normal_1, normal_2)
    omega_1 = angle_between_planes(normal_2, normal_3)
    phi_2 = angle_between_planes(normal_3, normal_4)

    return psi_1, omega_1, phi_2

def add_none_residues(residues):
    """adding None in place of residues which are not present"""
    max_res_num = 0
    for r in residues:
        max_res_num = max(max_res_num, r.residue_num)
    
    result = [None]*max_res_num
    
    for r in residues:
        result[r.residue_num-1] = r

    return result 


In [12]:
#  residues
residues = utils.pdb2Residues(model)

# getting chain A
residues = [r for r in residues if r.chain_identifier is not None and r.chain_identifier == 'A']

# removing coded residues
residues = [r for r in residues if r.code == '']

residues = add_none_residues(residues)


In [23]:
result = [{'phi': None, 'psi':None, 'omega': None}  for i in range(len(residues))]

for i in range(len(residues)-1):
    psi_1, omega_1, phi_2 = get_dihedral_angles(residues[i], residues[i+1])
    result[i]['psi'] = psi_1
    result[i]['omega'] = omega_1
    result[i+1]['phi'] = phi_2

result = [{residues[i].residue: result[i]} if residues[i] is not None else {None: result[i]} for i in range(len(result)) ]

In [24]:
result

[{'GLU': {'phi': None,
   'psi': np.float64(20.41760886060924),
   'omega': np.float64(178.65166633575367)}},
 {'MET': {'phi': np.float64(95.81191309668044),
   'psi': np.float64(21.723663959491425),
   'omega': np.float64(179.83920460853412)}},
 {'VAL': {'phi': np.float64(61.15303785752232),
   'psi': np.float64(136.25911028038573),
   'omega': np.float64(179.46420860403623)}},
 {'ASP': {'phi': np.float64(73.24853466120368),
   'psi': np.float64(26.20766040694992),
   'omega': np.float64(176.21069171628474)}},
 {'ASN': {'phi': np.float64(94.51746741317666),
   'psi': np.float64(8.693440007527554),
   'omega': np.float64(176.38546283000434)}},
 {'LEU': {'phi': np.float64(115.12477027974673),
   'psi': np.float64(138.67785060777595),
   'omega': np.float64(174.44754749663437)}},
 {'ARG': {'phi': np.float64(123.0742065142091),
   'psi': np.float64(172.09624534186483),
   'omega': np.float64(179.2273174502387)}},
 {'GLY': {'phi': np.float64(106.97066979724441),
   'psi': np.float64(171.26

In [12]:
result = [{'phi': None, 'psi':None, 'omega': None}  for i in range(len(residues))]

for i in range(len(residues)-1):
    psi_1, omega_1, phi_2 = get_dihedral_angles(residues[i], residues[i+1])
    result[i]['psi'] = psi_1
    result[i]['omega'] = omega_1
    result[i+1]['phi'] = phi_2

In [20]:
t = time.time()

for i in range(1000):
    with open('3k5d.pdb') as f:
        pdb = f.read()
    model = utils.getModel(pdb)

    #  residues
    residues = utils.pdb2Residues(model)

    # getting chain A
    residues = [r for r in residues if r.chain_identifier is not None and r.chain_identifier == 'A']

    # removing codeed residues
    residues = [r for r in residues if r.code == '']

    result = [{'phi': None, 'psi':None, 'omega': None}  for i in range(len(residues))]

    for i in range(len(residues)-1):
        psi_1, omega_1, phi_2 = get_dihedral_angles(residues[i], residues[i+1])
        result[i]['psi'] = psi_1
        result[i]['omega'] = omega_1
        result[i+1]['phi'] = phi_2

In [19]:
import mdtraj as md
import numpy as np
import time

t = time.time()

for i in range(1000):
    # Load PDB file
    pdb_file = "3k5d.pdb"  # Replace with your file
    traj = md.load(pdb_file)

    # Extract backbone dihedral angles
    phi_angles = md.compute_phi(traj)[1]  # ϕ angles
    psi_angles = md.compute_psi(traj)[1]  # ψ angles
    omega_angles = md.compute_omega(traj)[1]  # ω angles

    # Extract side-chain dihedral angles
    chi1_angles = md.compute_chi1(traj)[1]  # χ1 angles
    chi2_angles = md.compute_chi2(traj)[1]  # χ2 angles

    # Convert to degrees
    phi_angles = np.degrees(phi_angles)
    psi_angles = np.degrees(psi_angles)
    omega_angles = np.degrees(omega_angles)
    chi1_angles = np.degrees(chi1_angles)
    chi2_angles = np.degrees(chi2_angles)

print(time.time() - t)




74.1318461894989


In [18]:
for i in range(10):
    print(result[i]['omega'], end = ' ')

178.65166633575367 179.83920460853412 179.46420860403623 176.21069171628474 176.38546283000434 174.44754749663437 179.2273174502387 174.70354746467635 178.08249926372147 179.50652662080074 